In [1]:
import numpy as np
import opendatasets as od
import pickle
import os
from pathlib import Path
from music21 import note , chord , stream , instrument , converter   

In [4]:
# username: gabriel404guietdupre
# key: 97d328e7f389ab6055fc7b9dd41327ea

if not os.path.exists("./Data/"):
    os.makedirs('./Data')
dataset = "https://www.kaggle.com/datasets/zakarii/lofi-hip-hop-midi"
od.download(dataset, data_dir='./Data/')

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: gabriel404guietdupre
Your Kaggle Key: ········


100%|██████████████████████████████████████| 37.8k/37.8k [00:00<00:00, 14.0MB/s]

In [20]:
midi = converter.parse('./Data/lofi-hip-hop-midi/Piano Chords MIDI.mid')
type(midi)

music21.stream.base.Score

In [22]:
midi.show('text')

{0.0} <music21.stream.Part 0x7f87e4ebdfd0>
    {0.0} <music21.stream.Measure 1 offset=0.0>
        {0.0} <music21.instrument.Instrument 'Ultra Soft Upright - Pattern 1: '>
        {0.0} <music21.clef.TrebleClef>
        {0.0} <music21.meter.TimeSignature 4/4>
        {0.0} <music21.stream.Voice 0x7f87e4fc9e50>
            {0.0} <music21.note.Rest dotted-eighth>
            {0.75} <music21.chord.Chord E-3 F#3 B-3>
            {3.0} <music21.note.Rest quarter>
        {0.0} <music21.stream.Voice 0x7f87e4fc9eb0>
            {0.0} <music21.note.Rest 1.25ql>
            {1.25} <music21.note.Note C#>
            {3.25} <music21.note.Rest dotted-eighth>
    {4.0} <music21.stream.Measure 2 offset=4.0>
        {0.0} <music21.stream.Voice 0x7f87e4fc9a90>
            {0.0} <music21.note.Rest dotted-eighth>
            {0.75} <music21.chord.Chord F3 G#3 C4>
            {3.0} <music21.note.Rest dotted-eighth>
            {3.75} <music21.chord.Chord B-4 C#5 F5>
        {0.0} <music21.stream.Voice 0x

In [23]:
notes_to_parse = midi.flat.notes
print(len(notes_to_parse))

16


In [25]:
for element in notes_to_parse:
    print(element, element.offset)   # offset is where is the note in the song

<music21.chord.Chord E-3 F#3 B-3> 0.75
<music21.note.Note C#> 1.25
<music21.chord.Chord F3 G#3 C4> 4.75
<music21.note.Note E-> 5.25
<music21.chord.Chord B-4 C#5 F5> 7.75
<music21.chord.Chord B-3 C#4 F4> 7.75
<music21.chord.Chord B-4 C#5 F5> 8.0
<music21.chord.Chord B-3 C#4 F4> 8.0
<music21.note.Note G#> 8.25
<music21.note.Note G#> 8.25
<music21.chord.Chord F4 G#4 C5> 11.75
<music21.chord.Chord F3 G#3 C4> 11.75
<music21.note.Note E-> 12.0
<music21.chord.Chord F4 G#4 C5> 12.0
<music21.chord.Chord F3 G#3 C4> 12.0
<music21.note.Note E-> 12.25


In [17]:
notes_to_parse[0]

<music21.chord.Chord B2 F#3 D4 A4 C#5>

In [29]:
# Pitch refers to the frequency of the sound, or how high or low a particular note is 
# and is represented with the letters [A, B, C, D, E, F, G], with A being the highest and G being the lowest
notes_to_parse[1].pitch , str(notes_to_parse[1].pitch)

(<music21.pitch.Pitch C#4>, 'C#4')

In [33]:
# Represent each chords and notes by integer
notes_to_parse[5].normalOrder

e <music21.chord.Chord B-3 C#4 F4>


[10, 1, 5]

In [34]:
notes_demo = []

for element in notes_to_parse:
    
    # if the element is a Note , then store it's Pitch
    if isinstance(element , note.Note):
        notes_demo.append(str(element.pitch))
        
    # if the element is a Chord , split each of the note of the chord and join them with +
    elif isinstance(element , chord.Chord):
        notes_demo.append('+'.join(str(n) for n in element.normalOrder))

In [35]:
len(notes_demo)

16

In [36]:
print(notes_demo[0:])

['3+6+10', 'C#4', '5+8+0', 'E-4', '10+1+5', '10+1+5', '10+1+5', '10+1+5', 'G#5', 'G#4', '5+8+0', '5+8+0', 'E-5', '5+8+0', '5+8+0', 'E-4']


In [44]:
# Get all the notes and chords from the midi files in the ./midi_songs directory 
notes = []
p = Path("./Data/lofi-hip-hop-midi/")

for file in p.glob("*.mid"):
    midi = converter.parse(file)
    # print(f"parsing {file}" , end = "  ")
    
    elements_to_parse = midi.flat.notes
    # print(f"length {len(elements_to_parse)}")
    
    for element in elements_to_parse:
        
        # if the element is a Note, then store it's Pitch
        if isinstance(element , note.Note):
            notes.append(str(element.pitch))
            
        # if the element is a Chord , then split each of the note and join with +
        elif isinstance(element , chord.Chord):
            notes.append("+".join(str(n) for n in element.normalOrder))

In [46]:
with open("./Data/notes" , "wb") as file:
    pickle.dump(notes , file)

In [47]:
with open("./Data/notes" , "rb") as file:
    notes = pickle.load(file)

In [48]:
print("Total notes: " , len(notes))
print("Unique notes: " , len(set(notes)))

Total notes:  2044
Unique notes:  257


In [49]:
n_vocab = len(set(notes))

In [50]:
# get all pitch names (unique classes)
pitchnames = sorted(set(notes))

# create a dictionary to map pitches to integers
note_to_int = dict((element , idx) for idx , element in enumerate(pitchnames))

# create a reverse mapping
int_to_note = {idx:element for element , idx in note_to_int.items()}

assert len(note_to_int) == n_vocab

In [54]:
# How many elements LSTM input should consider
sequence_len = 100

In [55]:
network_input = []     # input sequence data
network_output = []    # output data

for i in range(len(notes) - sequence_len):
    seq_in = notes[i : i+sequence_len]         # contains 100 values
    seq_out = notes[i+sequence_len]
    
    network_input.append([note_to_int[n] for n in seq_in])
    network_output.append(note_to_int[seq_out])

In [57]:
len(network_input) , len(network_output)

(1944, 1944)

In [59]:
np.asarray(network_input).shape

(1944, 100)

In [63]:
# reshape input data into a shape compatible with LSTM layers
normalised_network_input = np.reshape(network_input , (*(np.asarray(network_input).shape) , 1))  # input_samples, sequence_len, 1
print(normalised_network_input.shape)

(1944, 100, 1)


In [64]:
normalised_network_input = normalised_network_input/float(n_vocab)

In [65]:
# Network output are the classes, so encode into one hot vector
from tensorflow.keras.utils import to_categorical
network_output = to_categorical(network_output)

2023-01-16 11:15:15.315415: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-16 11:15:15.315479: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [70]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [71]:
model = Sequential()
model.add(LSTM(units = 512 , input_shape = (normalised_network_input.shape[1], normalised_network_input.shape[2])
               , return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 512 , return_sequences = True))
model.add(Dropout(0.3))

model.add(LSTM(units = 512))
model.add(Dense(256))
model.add(Dropout(0.3))

model.add(Dense(n_vocab , activation = 'softmax'))

2023-01-16 11:19:18.809883: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-01-16 11:19:18.810035: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (fedora): /proc/driver/nvidia/version does not exist
2023-01-16 11:19:18.911733: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [72]:
model.compile(loss = "categorical_crossentropy", optimizer = "adam")
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 dropout (Dropout)           (None, 100, 512)          0         
                                                                 
 lstm_1 (LSTM)               (None, 100, 512)          2099200   
                                                                 
 dropout_1 (Dropout)         (None, 100, 512)          0         
                                                                 
 lstm_2 (LSTM)               (None, 512)               2099200   
                                                                 
 dense (Dense)               (None, 256)               131328    
                                                                 
 dropout_2 (Dropout)         (None, 256)               0

In [74]:
checkpoint = ModelCheckpoint("weights.h5", monitor = 'loss', save_best_only=True, mode = 'min')
hist = model.fit(normalised_network_input, network_output, epochs = 50, batch_size = 64, callbacks = [checkpoint])

Epoch 1/50
31/31 [==============================] - 100s 3s/step - loss: 5.1204
Epoch 2/50
31/31 [==============================] - 116s 4s/step - loss: 4.9740
Epoch 3/50
31/31 [==============================] - 110s 4s/step - loss: 4.9526
Epoch 4/50
31/31 [==============================] - 111s 4s/step - loss: 4.9346
Epoch 5/50
31/31 [==============================] - 110s 4s/step - loss: 4.9268
Epoch 6/50
31/31 [==============================] - 112s 4s/step - loss: 4.9126
Epoch 7/50
31/31 [==============================] - 116s 4s/step - loss: 4.8983
Epoch 8/50
31/31 [==============================] - 117s 4s/step - loss: 4.8609
Epoch 9/50
31/31 [==============================] - 110s 4s/step - loss: 4.8205
Epoch 10/50
31/31 [==============================] - 110s 4s/step - loss: 4.7635
Epoch 11/50
31/31 [==============================] - 118s 4s/step - loss: 4.7236
Epoch 12/50
23/31 [=====================>........] - ETA: 29s - loss: 4.6520

KeyboardInterrupt: 